In [1]:
from src.pdf_loader import load_pdf
from src.chunker import split_text_into_documents
from src.indexer import MultiIndex
from src.generator import generate_answer

# Indexes creation

Define indexes parameters

In [2]:
pdf_folder = "books"
pdf_files = {
    "Harry Potter 1": f"{pdf_folder}/harry-potter-1.pdf",
    "Harry Potter 2": f"{pdf_folder}/harry-potter-2.pdf",
    "Hunger Games": f"{pdf_folder}/HungerGames-1.pdf"
}

chunk_size = 500 #Number of characters in a chunk
chunk_overlap = 50 #Number of characters to overlap between two chunks to not cut a sentence in half.

In [3]:
def build_and_save_indexes(pdf_files, chunk_size=500, chunk_overlap=50):
    all_documents = []

    for source, filepath in pdf_files.items():
        print(f"Creating index on {source}...")
        texts, page_map = load_pdf(filepath, source)
        if texts:
            docs = split_text_into_documents(texts, source, page_map, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
            all_documents.extend(docs)
        else:
            print(f"Error when loading {filepath}.")

    print(f"Number of chunks : {len(all_documents)}")

    index_manager = MultiIndex()
    index_manager.build_indexes(all_documents)
    index_manager.save_indexes()

build 3 different indexes for the 3 books.
At first a unique index on all books was created. But the model could give inconsistent answers on cross books questions as the number of extracts from the different books in the prompt wasn't balanced enough (Bias toward HP1 extracts for exemple).
Solution is to create an index per book, with a maximum number of extracts selected per book to keep balance.

In [4]:
build_and_save_indexes(pdf_files, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

Creating index on Harry Potter 1...
Creating index on Harry Potter 2...
Creating index on Hunger Games...
Number of chunks : 3639


c:\Users\Gatsby\Documents\test_technique_myteam\retrieval_augmented_generation_from_pdf_books\retrieval_augmented_generation_from_pdf_books\src\indexer.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  self.embedding_model = HuggingFaceEmbeddings(model_name=model_name)
C:\Users\Gatsby\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Building index for Harry Potter 1 with 1123 chunks...
Building index for Harry Potter 2 with 1220 chunks...
Building index for Hunger Games with 1296 chunks...
Index saved: faiss_indexes\Harry_Potter_1
Index saved: faiss_indexes\Harry_Potter_2
Index saved: faiss_indexes\Hunger_Games


# Get answers to your queries

Define questions

In [5]:
queries = {
    "Harry Potter": [
        "À quelle heure le train 'Poudlard Express' est parti de la gare ?",
        "Pourquoi Hermione s'est enfermée dans les toilettes des filles ?",
        "Qui est Firenze et comment a-t-il aidé Harry ?",
        "Comment s'appelle l'ami roux de Harry ?",
    ],
    "Hunger Games": [
        "Quel est le nom de la soeur de Katniss Everdeen dans Hunger Games ?",
    ],
    "Cross-livres": [
        "Quel est le point commun entre le père d'Harry Potter et le père du personnage de Hunger Games ?",
        "Qui est le plus âgé des personnages principaux entre Harry Potter et Katniss Everdeen ?",
    ]
}

In [6]:
def get_answers_to_queries(queries, nb_extracts = 20, max_extracts_per_book = 7, model = "gpt-4o-mini", temperature = 0.0):
    indexer = MultiIndex()
    indexer.load_indexes()

    print("Indexes loaded successfully")
    
    answers = {}
    
    for query in queries:
        retrieved_docs = indexer.search(query, nb_extracts=nb_extracts, max_extracts_per_book=max_extracts_per_book)

        print(f"\nQuestion: {query}")
        print("Extracts found:")

        for doc in retrieved_docs:
            print(f"- Source : {doc.metadata.get('source')} | Page : {doc.metadata.get('page_start')}")
            print(doc.page_content[:300], "...\n")

        answer = generate_answer(query, retrieved_docs, model=model, temperature=temperature)
        answers[query] = answer
    
    return answers


Define model parameters

In [7]:
nb_extracts = 20 #Number of selected chunks for each prompt
max_extracts_per_book = 7 #Maximum number of chunks for one book

model="gpt-4o-mini" #Used model
temperature=0.0 #Temperature of the model. 0 gives the most consistent answers with the least halucinations.

Generate all answers

In [8]:
all_answers = {}

for book, questions in queries.items():
    all_answers[book] = get_answers_to_queries(questions)

Loading index for Harry Potter 1...
Loading index for Harry Potter 2...
Loading index for Hunger Games...
Indexes loaded successfully

Question: À quelle heure le train 'Poudlard Express' est parti de la gare ?
Extracts found:
- Source : Hunger Games | Page : 185
m’échapper d’ici, de repousser Cato ou Thresh en escaladant ce terrain
rocailleux. En fait, je suis en train de me dire que j’ai fait fausse route, qu’un
garçon blessé n’aurait pas pu se réfugier par là, quand je repère du sang sur
un rocher. Il a séché depuis longtemps, mais les traînées parallèles ...

- Source : Hunger Games | Page : 149
— Oui, absolument. J’imagine Haymitch en train de s’arracher les
cheveux en me voyant faire équipe avec cette gamine. Mais je le veux. C’est
une survivante, j’ai confiance en elle, et – pourquoi ne pas l’admettre ? – elle
me fait penser à Prim.
— D’accord, dit-elle en tendant le bras. (On se serre l ...

- Source : Harry Potter 2 | Page : 82
portes	en	bois	des	cabines	étaient	écaillées	et	l

Look at the results.

In [9]:
for book, answers in all_answers.items():
    print(f"Questions on {book}")
    for query, answer in answers.items():
        print(f"{query} : \n{answer}")
        print("------------------------------------------------")
        print()

Questions on Harry Potter
À quelle heure le train 'Poudlard Express' est parti de la gare ? : 
**Réponse :** Le train 'Poudlard Express' est parti de la gare à 11 heures.

**Sources utilisées :**
- Livre : Harry Potter 1, Page : 65
------------------------------------------------

Pourquoi Hermione s'est enfermée dans les toilettes des filles ? : 
**Réponse :** Hermione s'est enfermée dans les toilettes des filles pour pleurer tout à son aise, car elle n'avait pas d'amis et se sentait mal.

**Sources utilisées :**
- Livre : Harry Potter 1, Page : 118
------------------------------------------------

Qui est Firenze et comment a-t-il aidé Harry ? : 
**Réponse :** Firenze est un centaure qui, sous le coup de la colère, se met à ruer et aide Harry en le portant sur son dos pour fuir un danger. Il lui demande s'il ne voit pas la licorne et lui explique qu'il se dresse contre ce qui se cache dans la forêt, même s'il doit venir en aide à un humain.

**Sources utilisées :**
- Livre : Harry Po